In [58]:
%load_ext autoreload
%autoreload 2
from utils import *
from KeywordExtractor import *
from gensim.models import Word2Vec
import networkx as nx
from itertools import combinations
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
#prune_text("aaa bbbb ccc")
abstract = load_abstract('ex1')
abstract = "The country of Japan has developed to a great economy after WW1. Japan is a developed island nation. I love."
ke = KeywordExtractor(abstract=abstract)
print(ke.tokens)
print(ke.sentences)
print(ke.graph)
ke.graph.nodes

['country', 'japan', 'developed', 'great', 'economy', 'ww1', 'japan', 'developed', 'island', 'nation', 'love']
[['country', 'japan', 'developed', 'great', 'economy', 'ww1'], ['japan', 'developed', 'island', 'nation'], ['love']]
Graph with 9 nodes and 0 edges


NodeView(('country', 'japan', 'developed', 'great', 'economy', 'ww1', 'island', 'nation', 'love'))

In [28]:
get_co_matrix(ke.sentences, ke.unique_tokens)
my_list = ['japan', 'love', 'economy']
p = list(combinations(my_list, 2))
print(p)

[('japan', 'love'), ('japan', 'economy'), ('love', 'economy')]


In [64]:
print(ke.sentences)
print(ke.unique_tokens)

co_matrix, index_dict = get_co_matrix(ke.sentences, ke.unique_tokens)
print(index_dict)
for c in co_matrix: print(c)
co_matrix[0][0]

[['country', 'japan', 'developed', 'great', 'economy', 'ww1'], ['japan', 'developed', 'island', 'nation'], ['love']]
['japan', 'love', 'economy', 'ww1', 'country', 'nation', 'great', 'developed', 'island']
['country', 'japan', 'developed']
['japan', 'developed', 'great']
['developed', 'great', 'economy']
['great', 'economy', 'ww1']
['japan', 'developed', 'island']
['developed', 'island', 'nation']
['love']
{'japan': 0, 'love': 1, 'economy': 2, 'ww1': 3, 'country': 4, 'nation': 5, 'great': 6, 'developed': 7, 'island': 8}
[0. 0. 0. 0. 1. 0. 1. 3. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 2. 1. 0.]
[0. 0. 1. 0. 0. 0. 1. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 1.]
[1. 0. 2. 1. 0. 0. 0. 2. 0.]
[3. 0. 1. 0. 1. 1. 2. 0. 2.]
[1. 0. 0. 0. 0. 1. 0. 2. 0.]


0.0